In [1]:
# ==============================================================================
# AURA AI ENGINE - ADIM 4 (GELİŞMİŞ): DERİNLEMESİNE İNCE AYAR
# ==============================================================================
# AMAÇ: Modelin son katmanlarını da eğitime dahil ederek ve öğrenme oranı
#       zamanlaması kullanarak modelin performansını artırmak.
# ------------------------------------------------------------------------------

# --- ADIM 1: Gerekli Kütüphanelerin Yüklenmesi ve Parametrelerin Ayarlanması ---

import torch
import torch.nn as nn
import torch.optim as optim
# YENİ: Zamanlayıcı için import eklendi
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import pandas as pd
import os
from PIL import Image
import time

# -- Ayarlar --
BATCH_SIZE = 32
LEARNING_RATE = 0.001
# Daha derin bir eğitim için epoch sayısını 10'da tutuyoruz.
NUM_EPOCHS = 10
NUM_CLASSES = 10

# Veri yolları (Notebook için '../' kullanılır, .py scripti için kaldırılır)
PROCESSED_DATA_PATH = os.path.join("..", "data", "processed")
IMAGES_PATH = os.path.join("..", "data", "raw", "images")
MODEL_SAVE_PATH = os.path.join("..", "models")
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılacak Cihaz: {device}")


# --- ADIM 2: Özel Veri Seti Sınıfı (Değişiklik Yok) ---

class FashionDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform
        self.classes = self.annotations['articleType'].astype('category').cat.categories
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.annotations['label'] = self.annotations['articleType'].apply(lambda x: self.class_to_idx[x])

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_id = self.annotations.iloc[idx, 0]
        img_path = os.path.join(self.image_dir, str(img_id) + ".jpg")
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(self.annotations.iloc[idx, -1], dtype=torch.long)
        
        if self.transform:
            image = self.transform(image)
            
        return image, label


# --- ADIM 3: Veri Dönüşümleri ve Yükleyiciler (Değişiklik Yok, Güçlü Augmentation Kalıyor) ---

data_transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.2))
])

data_transforms_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = FashionDataset(csv_file=os.path.join(PROCESSED_DATA_PATH, "train_cleaned.csv"),
                               image_dir=IMAGES_PATH,
                               transform=data_transforms_train)

val_dataset = FashionDataset(csv_file=os.path.join(PROCESSED_DATA_PATH, "validation_cleaned.csv"),
                             image_dir=IMAGES_PATH,
                             transform=data_transforms_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
print("Veri Yükleyiciler hazır.")


# --- ADIM 4: Modelin Özelleştirilmesi (YENİ - Katmanlar Çözüldü) ---

model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False

# YENİ: Son konvolüsyonel bloğun (layer4) parametrelerini çözüyoruz.
# Artık bu katman da eğitim sırasında güncellenecek.
for param in model.layer4.parameters():
    param.requires_grad = True

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES) # Bu katmanın requires_grad'ı zaten True

model = model.to(device)
print("Model özelleştirildi, son katman bloğu çözüldü ve cihaza taşındı.")


# --- ADIM 5: Loss Fonksiyonu, Optimizer ve Zamanlayıcı (YENİ) ---

criterion = nn.CrossEntropyLoss()

# YENİ: Optimizer'ı iki farklı parametre grubunu, farklı öğrenme oranlarıyla
# güncelleyecek şekilde tanımlıyoruz.
params_to_update = [
    # Yeni eklediğimiz fc katmanı daha hızlı öğrenmeli
    {'params': model.fc.parameters(), 'lr': LEARNING_RATE},
    # Çözdüğümüz layer4 ise daha yavaş öğrenerek eski bilgilerini korumalı
    {'params': model.layer4.parameters(), 'lr': LEARNING_RATE / 10}
]
optimizer = optim.Adam(params_to_update)

# YENİ: Öğrenme oranı zamanlayıcısını tanımlıyoruz.
# Her 3 epoch'ta bir, öğrenme oranını 0.1 ile çarpacak (yani 10'a bölecek).
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)


# --- ADIM 6: Eğitim ve Doğrulama Döngüsü (YENİ - scheduler.step() eklendi) ---

print("\n--- GELİŞMİŞ EĞİTİM BAŞLIYOR ---")
start_time = time.time()
best_val_acc = 0.0 # En iyi modeli doğruluk oranına göre seçeceğiz

for epoch in range(NUM_EPOCHS):
    
    # ... (Eğitim ve Doğrulama aşamaları aynı kalıyor) ...
    model.train()
    running_loss = 0.0
    running_corrects = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)
    
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)
            
    val_loss = val_loss / len(val_dataset)
    val_acc = val_corrects.double() / len(val_dataset)
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | "
          f"Eğitim Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} | "
          f"Doğrulama Loss: {val_loss:.4f} Acc: {val_acc:.4f}")
    
    # YENİ: Her epoch'un sonunda zamanlayıcıyı güncelliyoruz.
    scheduler.step()
    
    # En iyi modeli doğruluk oranına (accuracy) göre kaydet
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), os.path.join(MODEL_SAVE_PATH, "best_model_advanced.pth"))
        print(f"** Modelin daha iyi bir versiyonu kaydedildi (Doğrulama Acc: {best_val_acc:.4f}) **")

end_time = time.time()
training_time = end_time - start_time
print("\n--- EĞİTİM TAMAMLANDI ---")
print(f"Toplam Eğitim Süresi: {training_time // 60:.0f} dakika {training_time % 60:.0f} saniye")
print(f"En iyi model 'models/best_model_advanced.pth' olarak kaydedildi.")

Kullanılacak Cihaz: cuda
Veri Yükleyiciler hazır.
Model özelleştirildi, son katman bloğu çözüldü ve cihaza taşındı.

--- GELİŞMİŞ EĞİTİM BAŞLIYOR ---
Epoch 1/10 | Eğitim Loss: 0.4638 Acc: 0.8279 | Doğrulama Loss: 0.2240 Acc: 0.9058
** Modelin daha iyi bir versiyonu kaydedildi (Doğrulama Acc: 0.9058) **
Epoch 2/10 | Eğitim Loss: 0.2661 Acc: 0.8967 | Doğrulama Loss: 0.1884 Acc: 0.9283
** Modelin daha iyi bir versiyonu kaydedildi (Doğrulama Acc: 0.9283) **
Epoch 3/10 | Eğitim Loss: 0.2241 Acc: 0.9140 | Doğrulama Loss: 0.1819 Acc: 0.9277
Epoch 4/10 | Eğitim Loss: 0.1852 Acc: 0.9273 | Doğrulama Loss: 0.1560 Acc: 0.9380
** Modelin daha iyi bir versiyonu kaydedildi (Doğrulama Acc: 0.9380) **
Epoch 5/10 | Eğitim Loss: 0.1696 Acc: 0.9329 | Doğrulama Loss: 0.1573 Acc: 0.9387
** Modelin daha iyi bir versiyonu kaydedildi (Doğrulama Acc: 0.9387) **
Epoch 6/10 | Eğitim Loss: 0.1684 Acc: 0.9329 | Doğrulama Loss: 0.1554 Acc: 0.9413
** Modelin daha iyi bir versiyonu kaydedildi (Doğrulama Acc: 0.9413) *